## A* search --- informed search method 
- This method comes under the heuristic algorithms.
- Uses h(n) + g(n) to choose between two choices available to perform action
- where h(n) is manhattan distance from the goal , g(n) is the total cost needed to reach at that state.
- A* handles the cases where manhattan or say greedyBFS struggles to find the optimal path.

In [5]:
maze5 = [
    ['A', 0,  1,  0,  0,  0, 1, 0, 0, 1, 0, 0],
    [0,   0,  1,  0,  1,  0, 1, 0, 1, 0, 0, 0],
    [1,   0,  0,  0,  1,  0, 0, 0, 1, 1, 1, 0],
    [0,   1,  1,  0,  0,  1, 1, 0, 0, 0, 1, 0],
    [0,   0,  0,  0,  1,  0, 1, 1, 1, 0, 0, 0],
    [1,   1,  1,  0,  1,  0, 0, 0, 1, 1, 1, 0],
    [0,   0,  1,  0,  0,  0, 1, 0, 0, 0, 1, 0],
    [0,   1,  0,  1,  1,  0, 1, 1, 1, 0, 1, 0],
    [0,   0,  0,  0,  0,  0, 0, 0, 1, 0, 0, 0],
    [1,   1,  1,  1,  1,  1, 0, 1, 0, 1, 1, 0],
    [0,   0,  0,  0,  0,  0, 0, 1, 0, 0, 0, 0],
    [0,   1,  1,  1,  1,  1, 1, 1, 1, 1, 1,'B'],
]


In [6]:
maze = [
    ['A', 0,  0,  0,  1,  1,  1,  1,  1,  1],
    [1,  1,  1,  0,  1,  0,  0,  0,  0,  1],
    [1,  0,  0,  0,  1,  0,  1,  1,  0,  1],
    [1,  0,  1,  1,  1,  0,  1,  0,  0,  0],
    [1,  0,  0,  0,  0,  0,  1,  0,  1, 'B'],
    [1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
]


## Code to solve the maze by A* approaach and to display the optimal path. 

In [7]:
import heapq

# --- Style and Color Definitions ---
class Colors:
    """A class to hold ANSI color codes for styling terminal output."""
    RESET = '\033[0m'
    BOLD = '\033[1m'
    DIM = '\033[2m'
    
    # Text Colors
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    WHITE = '\033[97m'

# --- The Maze Definition ---

# --- The Complete MazeSolver Class ---
class MazeSolver:
    def __init__(self, maze):
        self.maze = maze
        self.start, self.goal = self.find_positions(maze)

        print("\n--- Manhattan Heuristic Visualization (Distance to 'B') ---")
        self.manhattanMaze = self.visualize_manhattan()
        self.print_matrix_styled(self.manhattanMaze)
        
        # Solve the maze using the heuristic
        self.path = self.a_star()
        
        if self.path:
            print("--- Solved Path ---")
            self.updated_manhattan(self.path)
            self.print_matrix_styled(self.manhattanMaze)

    def manhattan(self, a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])
    
    def get_neighbors(self, pos):
        neighbors = []
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # up, down, left, right
        rows, cols = len(self.maze), len(self.maze[0])
        for dx, dy in directions:
            nx, ny = pos[0] + dx, pos[1] + dy
            if 0 <= nx < rows and 0 <= ny < cols and self.maze[nx][ny] != 1:
                neighbors.append((nx, ny))
        return neighbors
    
    def find_positions(self, maze):
        start = goal = None
        for i, row in enumerate(maze):
            for j, cell in enumerate(row):
                if cell == 'A':
                    start = (i, j)
                elif cell == 'B':
                    goal = (i, j)
        return start, goal

    def a_star(self):
        if not self.start or not self.goal:
            print("Start or goal not found in the maze.")
            return []

        visited = set()
        came_from = {}
        heap = []
        heapq.heappush(heap, (self.manhattan(self.start, self.goal), self.start,0))

        while heap:
            # print("Heap:", heap)
            _, current,state= heapq.heappop(heap)

            if current == self.goal:
                break

            if current in visited:
                continue
            visited.add(current)
            # start += 1
            for neighbor in self.get_neighbors(current):
                if neighbor not in visited:
                    priority = self.manhattan(neighbor, self.goal)
                    heapq.heappush(heap, (priority+state+1,neighbor,state+1))
                    came_from[neighbor] = current
        
        path = []
        curr = self.goal
        while curr != self.start:
            path.append(curr)
            curr = came_from.get(curr)
            if curr is None:
                print("No path found.")
                return []
        path.append(self.start)
        path.reverse()
        return path
    
    def visualize_manhattan(self):
        rows, cols = len(self.maze), len(self.maze[0])
        dist_matrix = []
        for i in range(rows):
            row = []
            for j in range(cols):
                if self.maze[i][j] == 1:
                    row.append("█")
                elif (i, j) == self.start:
                    row.append('A')
                elif (i, j) == self.goal:
                    row.append('B')
                else:
                    row.append(self.manhattan((i, j), self.goal))
            dist_matrix.append(row)
        return dist_matrix

    def updated_manhattan(self, path):
        for (x, y) in path:
            if self.manhattanMaze[x][y] not in ('A', 'B'):
                self.manhattanMaze[x][y] = '*'
    
    def print_matrix_styled(self, matrix):
        """
        Prints the maze matrix with a game-like border and colors.
        """
        rows, cols = len(matrix), len(matrix[0])

        print(f"{Colors.BOLD}{Colors.WHITE}╔{'═' * (cols * 3 + 1)}╗{Colors.RESET}")

        for row in matrix:
            print(f"{Colors.BOLD}{Colors.WHITE}║ {Colors.RESET}", end="")
            for val in row:
                cell_str = f"{str(val):<2}"
                
                if val == 'A':
                    print(f"{Colors.BOLD}{Colors.GREEN}{cell_str}{Colors.RESET} ", end="")
                elif val == 'B':
                    print(f"{Colors.BOLD}{Colors.YELLOW}{cell_str}{Colors.RESET} ", end="")
                elif val == '█':
                    print(f"{Colors.BLUE}{'█ '}{Colors.RESET} ", end="")
                elif val == '*':
                    print(f"{Colors.BOLD}{Colors.CYAN}{'• '}{Colors.RESET} ", end="")
                else: # Heuristic numbers
                    print(f"{Colors.DIM}{cell_str}{Colors.RESET} ", end="")
            
            print(f"{Colors.BOLD}{Colors.WHITE}║{Colors.RESET}")

        print(f"{Colors.BOLD}{Colors.WHITE}╚{'═' * (cols * 3 + 1)}╝{Colors.RESET}")
        print()



solver = MazeSolver(maze5)


--- Manhattan Heuristic Visualization (Distance to 'B') ---
╔═════════════════════════════════════╗
║ A  21 █  19 18 17 █  15 14 █  12 11 ║
║ 21 20 █  18 █  16 █  14 █  12 11 10 ║
║ █  19 18 17 █  15 14 13 █  █  █  9  ║
║ 19 █  █  16 15 █  █  12 11 10 █  8  ║
║ 18 17 16 15 █  13 █  █  █  9  8  7  ║
║ █  █  █  14 █  12 11 10 █  █  █  6  ║
║ 16 15 █  13 12 11 █  9  8  7  █  5  ║
║ 15 █  13 █  █  10 █  █  █  6  █  4  ║
║ 14 13 12 11 10 9  8  7  █  5  4  3  ║
║ █  █  █  █  █  █  7  █  5  █  █  2  ║
║ 12 11 10 9  8  7  6  █  4  3  2  1  ║
║ 11 █  █  █  █  █  █  █  █  █  █  B  ║
╚═════════════════════════════════════╝

--- Solved Path ---
╔═════════════════════════════════════╗
║ A  21 █  19 18 17 █  15 14 █  12 11 ║
║ •  •  █  18 █  16 █  14 █  12 11 10 ║
║ █  •  •  •  █  15 14 13 █  █  █  9  ║
║ 19 █  █  •  15 █  █  12 11 10 █  8  ║
║ 18 17 16 •  █  13 █  █  █  9  8  7  ║
║ █  █  █  •  █  •  •  •  █  █  █  6  ║
║ 16 15 █  •  •  •  █  •  •  •  █  5  ║
║ 15 █  13 █  █  10 █  █  █  •  █  4  

## More updated code
### Also display the routes taken in the A* search along with the optimal path.
- Easy to follow the A* search with the comparison of the optimal path and  other paths which has been checked in the search.

In [8]:
import heapq

class Colors:
    RESET = '\033[0m'
    BOLD = '\033[1m'
    DIM = '\033[2m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    WHITE = '\033[97m'

class MazeSolver:
    def __init__(self, maze):
        self.maze = maze
        self.start, self.goal = self.find_positions(maze)

        print("\n--- Manhattan Heuristic Visualization (Distance to 'B') ---")
        self.manhattanMaze = self.visualize_manhattan()
        self.print_matrix_styled(self.manhattanMaze)

        # Solve the maze and capture visited
        self.path, self.visited = self.greedy_bfs()

        if self.path:
            print("--- Solved Path with Visited Nodes ---")
            self.updated_manhattan(self.path, self.visited)
            self.print_matrix_styled(self.manhattanMaze)
        else:
            print("No path found.")

    def manhattan(self, a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    def get_neighbors(self, pos):
        neighbors = []
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # up, down, left, right
        rows, cols = len(self.maze), len(self.maze[0])
        for dx, dy in directions:
            nx, ny = pos[0] + dx, pos[1] + dy
            if 0 <= nx < rows and 0 <= ny < cols and self.maze[nx][ny] != 1:
                neighbors.append((nx, ny))
        return neighbors

    def find_positions(self, maze):
        start = goal = None
        for i, row in enumerate(maze):
            for j, cell in enumerate(row):
                if cell == 'A':
                    start = (i, j)
                elif cell == 'B':
                    goal = (i, j)
        return start, goal

    def greedy_bfs(self):
        if not self.start or not self.goal:
            print("Start or goal not found in the maze.")
            return []

        visited = set()
        came_from = {}
        heap = []
        heapq.heappush(heap, (self.manhattan(self.start, self.goal), self.start,0))

        while heap:
            # print("Heap:", heap)
            _, current,state= heapq.heappop(heap)

            if current == self.goal:
                break

            if current in visited:
                continue
            visited.add(current)
            for neighbor in self.get_neighbors(current):
                if neighbor not in visited:
                    priority = self.manhattan(neighbor, self.goal)
                    heapq.heappush(heap, (priority+state+1,neighbor,state+1))
                    came_from[neighbor] = current
        
        path = []
        curr = self.goal
        while curr != self.start:
            path.append(curr)
            curr = came_from.get(curr)
            if curr is None:
                print("No path found.")
                return []
        path.append(self.start)
        path.reverse()
        return path, visited

    def visualize_manhattan(self):
        rows, cols = len(self.maze), len(self.maze[0])
        dist_matrix = []
        for i in range(rows):
            row = []
            for j in range(cols):
                if self.maze[i][j] == 1:
                    row.append("█")
                elif (i, j) == self.start:
                    row.append('A')
                elif (i, j) == self.goal:
                    row.append('B')
                else:
                    row.append(self.manhattan((i, j), self.goal))
            dist_matrix.append(row)
        return dist_matrix

    def updated_manhattan(self, path, visited):
        for (x, y) in visited:
            if self.manhattanMaze[x][y] not in ('A', 'B', '*'):
                self.manhattanMaze[x][y] = '.'

        for (x, y) in path:
            if self.manhattanMaze[x][y] not in ('A', 'B'):
                self.manhattanMaze[x][y] = '*'

    def print_matrix_styled(self, matrix):
        rows, cols = len(matrix), len(matrix[0])
        print(f"{Colors.BOLD}{Colors.WHITE}╔{'═' * (cols * 3 + 1)}╗{Colors.RESET}")

        for row in matrix:
            print(f"{Colors.BOLD}{Colors.WHITE}║ {Colors.RESET}", end="")
            for val in row:
                cell_str = f"{str(val):<2}"
                if val == 'A':
                    print(f"{Colors.BOLD}{Colors.GREEN}{cell_str}{Colors.RESET} ", end="")
                elif val == 'B':
                    print(f"{Colors.BOLD}{Colors.YELLOW}{cell_str}{Colors.RESET} ", end="")
                elif val == '█':
                    print(f"{Colors.BLUE}{'█ '}{Colors.RESET} ", end="")
                elif val == '*':
                    print(f"{Colors.BOLD}{Colors.CYAN}{'• '}{Colors.RESET} ", end="")
                elif val == '.':
                    print(f"{Colors.BOLD}{Colors.YELLOW}{'. '}{Colors.RESET} ", end="")
                else:
                    print(f"{Colors.DIM}{cell_str}{Colors.RESET} ", end="")
            print(f"{Colors.BOLD}{Colors.WHITE}║{Colors.RESET}")
        print(f"{Colors.BOLD}{Colors.WHITE}╚{'═' * (cols * 3 + 1)}╝{Colors.RESET}")
        print()

solver = MazeSolver(maze)


--- Manhattan Heuristic Visualization (Distance to 'B') ---
╔═══════════════════════════════╗
║ A  12 11 10 █  █  █  █  █  █  ║
║ █  █  █  9  █  7  6  5  4  █  ║
║ █  10 9  8  █  6  █  █  3  █  ║
║ █  9  █  █  █  5  █  3  2  1  ║
║ █  8  7  6  5  4  █  2  █  B  ║
║ █  █  █  █  █  █  █  █  █  █  ║
╚═══════════════════════════════╝

--- Solved Path with Visited Nodes ---
╔═══════════════════════════════╗
║ A  •  •  •  █  █  █  █  █  █  ║
║ █  █  █  •  █  •  •  •  •  █  ║
║ █  •  •  •  █  •  █  █  •  █  ║
║ █  •  █  █  █  •  █  3  •  •  ║
║ █  •  •  •  •  •  █  2  █  B  ║
║ █  █  █  █  █  █  █  █  █  █  ║
╚═══════════════════════════════╝

